In [ ]:
# Jeśli jeszcze nie zainstalowany:
pip install pydub

In [1]:
import wave
from pydub import AudioSegment as auds
import pandas as pd
import numpy as np
import os, fnmatch
from matplotlib import pyplot as plt
import scipy.io.wavfile as wf
from scipy.integrate import simps

C:\Users\Lenovo\anaconda3\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [7]:
# Z kilkoma plikami jest problem, nie wszystkie słowa się eksportują, co powoduje błędy później w liczeniu parametrów
# Ten skrypt działa dla plików:
# 0 289516_25_K_01_2.txt 289516_25_K_01_2.wav
# 1 289516_25_K_10_3.txt 289516_25_K_10_3.wav
# 2 289516_25_K_16_4.txt 289516_25_K_16_4.wav
# 3 289516_25_K_22_1.txt 289516_25_K_22_1.wav
# 4 289540_23_K_13_2.txt 289540_23_K_13_2.wav
# 5 289540_23_K_18_3.txt 289540_23_K_18_3.wav
# 6 289540_23_K_22_4.txt 289540_23_K_22_4.wav
# 7 289540_23_K_9_1.txt 289540_23_K_9_1.wav
# 8 289550_23_K_18_2.txt 289550_23_K_18_2.wav
# 9 289550_23_K_23_3.txt 289550_23_K_23_3.wav
# 10 289550_23_K_23_4.txt 289550_23_K_23_4.wav
# 11 289550_23_K_9_1.txt 289550_23_K_9_1.wav
# 12 289556_24_K_11_1.txt 289556_24_K_11_1.wav
# 13 289556_24_K_15_2.txt 289556_24_K_15_2.wav
# 14 289556_24_K_19_3.txt 289556_24_K_19_3.wav
# 15 289556_24_K_23_4.txt 289556_24_K_23_4.wav
# 16 404907_23_K_11_1.txt 404907_23_K_11_1.wav
# 17 404907_23_K_14_2.txt 404907_23_K_14_2.wav

# Przynajmniej u mnie, jeśli w folderze ze ścieżki mam też inne pliki poza tymi wymienionymi,
# to w pewnym momencie liczenie się zacina, może to jakiś problem z mojej strony



In [9]:
path = 'C:/Dysk D/STUDIA/Eksploracja danych biomedycznych/Nagrania'

# Wyciaganie nazw plikow ze sciezki
list_of_files = os.listdir(path)
pattern = "*.txt"
list_of_names = []
for entry in list_of_files:
    if fnmatch.fnmatch(entry, pattern):
            list_of_names.append(entry[:-4])

# Utworzenie list plikow wav i txt
files_wav_list = []
files_txt_list = []
for filename in list_of_names:
    files_wav_list.append(filename + '.wav')
    files_txt_list.append(filename + '.txt')
    
print('Obecne pliki:')

for i in range(len(list_of_names)):
    print(i, files_txt_list[i], files_wav_list[i])
print('liczba plikow: ', len(list_of_names))

Obecne pliki:
0 289516_25_K_01_2.txt 289516_25_K_01_2.wav
1 289516_25_K_10_3.txt 289516_25_K_10_3.wav
2 289516_25_K_16_4.txt 289516_25_K_16_4.wav
3 289516_25_K_22_1.txt 289516_25_K_22_1.wav
4 289540_23_K_13_2.txt 289540_23_K_13_2.wav
5 289540_23_K_18_3.txt 289540_23_K_18_3.wav
6 289540_23_K_22_4.txt 289540_23_K_22_4.wav
7 289540_23_K_9_1.txt 289540_23_K_9_1.wav
8 289550_23_K_18_2.txt 289550_23_K_18_2.wav
9 289550_23_K_23_3.txt 289550_23_K_23_3.wav
10 289550_23_K_23_4.txt 289550_23_K_23_4.wav
11 289550_23_K_9_1.txt 289550_23_K_9_1.wav
12 289556_24_K_11_1.txt 289556_24_K_11_1.wav
13 289556_24_K_15_2.txt 289556_24_K_15_2.wav
14 289556_24_K_19_3.txt 289556_24_K_19_3.wav
15 289556_24_K_23_4.txt 289556_24_K_23_4.wav
16 404907_23_K_11_1.txt 404907_23_K_11_1.wav
17 404907_23_K_14_2.txt 404907_23_K_14_2.wav
liczba plikow:  18


In [8]:
# Liczenie parametrów zapisywanych do plików osób (w każdym pliku 98 wartości, kolejne słowa jednej osoby)

# Parametr: pole powierzchni pod krzywą z wartości bezwzględnej sygnału (area)
# Parametr: gęstość przejść przez 0 (zeros)

folder_name_area = path + '/' + 'Area_osoby' + '/' # tworzenie folderu na wyniki danego parametru
folder_name_zeros = path + '/' + 'Zeros_osoby' + '/'
for i in range(len(list_of_names)): # iteracja po 18 plikach
    print('i = ', i)
    filename = list_of_names[i]
    file_wav = files_wav_list[i]
    file_txt = files_txt_list[i]
    labels = []
    # Wczytywanie plikow z etykietami
    textfile = pd.read_csv(path + '/' + file_txt, sep = '\t', header = None)
    osoba = []
    print("Z nagrania:", filename)
    # Wczytywanie etykiet i odpowiadających im słów, zapisywanie do listy
    for j in range(98): # iteracja po kolejnych słowach z jednego pliku
        labels.append(str(textfile[2][j]))
        if labels[j] == 'nan':
            labels[j] = 'NA'
        labels[j] = labels[j].upper() 
        word_path = path + '/' + labels[j]
        # Sprawdź, czy istnieje plik o danej nazwie
        if os.path.isfile(word_path + '/' + filename + '_' + '{0}.wav'.format(labels[j])) == True:
            new_filename = filename + '_' + labels[j] + '.wav'
#             print(j)
            word = wave.open(word_path + '/' + new_filename, 'rb')
            # Jeśli ilość ramek nieparzysta, to nie zadziała, więc wtedy wczytujemy wszystkie ramki oprócz ostatniej
            if word.getnframes() % 2 != 0:
                word_frames = word.readframes(word.getnframes()-1)
            else:
                word_frames = word.readframes(word.getnframes())
            word_frames = np.frombuffer(word_frames, dtype=int)
            osoba.append(word_frames)
        
    # Operacje na liście wszystkich słów z danego nagrania
    area_person = []
    zeros_person = []
    for k in range(len(osoba)): # iteracja po kolejnych słowach
#         if k < 10:
#             print('k =',  k)
        # np. dodaj linijkę "folder_name_nazwa_parametru = path + '/' + 'Nazwa_parametru_osoby' + '/'"
        if os.path.isdir(folder_name_area) == False:
            os.mkdir(folder_name_area)
        if os.path.isdir(folder_name_zeros) == False:
            os.mkdir(folder_name_zeros)
#         print('Pierwsze 5 plików:')
        # for j in range(len(list_of_names)):
        zeros_cnt = 0
        area_person.append(simps(np.abs(osoba[k])))
        for m in range(1, len(osoba[k])):
            if (osoba[k][m] <= 0 and osoba[k][m-1] > 0) or (osoba[k][m] >= 0 and osoba[k][m-1] < 0):
                zeros_cnt += 1
    # >>> skopiuj linijkę powyżej i zmień na <nazwa_parametru_person>.append(<wyliczanie parametru>(osoba[i])) <<<
    # Zapis wartości do pliku tekstowego
        zeros_person.append(zeros_cnt)
#         print('k = ', k, 'Dodano gęstość przejść przez zero')
    txt_person_filename_area = folder_name_area + 'area_' + list_of_names[i] + '.txt'
    txt_person_filename_zeros = folder_name_zeros + 'zeros_' + list_of_names[i] + '.txt'
    if os.path.isfile(txt_person_filename_area) == False:
        np.savetxt(txt_person_filename_area, area_person)
        print('Zapisano jako:', txt_person_filename_area)
    else:
        print('Plik', list_of_names[i], 'z AREA już istnieje') 
    if os.path.isfile(txt_person_filename_zeros) == False:
        np.savetxt(txt_person_filename_zeros, zeros_person)
        print('Zapisano jako:', txt_person_filename_zeros)
    elif os.path.isfile(txt_person_filename_zeros) == True:
        print('Plik', list_of_names[i], 'z ZEROS już istnieje') 

i =  0
Z nagrania: 289516_25_K_01_2
Plik 289516_25_K_01_2 z AREA już istnieje
Plik 289516_25_K_01_2 z ZEROS już istnieje
i =  1
Z nagrania: 289516_25_K_10_3
Plik 289516_25_K_10_3 z AREA już istnieje
Plik 289516_25_K_10_3 z ZEROS już istnieje
i =  2
Z nagrania: 289516_25_K_16_4
Plik 289516_25_K_16_4 z AREA już istnieje
Plik 289516_25_K_16_4 z ZEROS już istnieje
i =  3
Z nagrania: 289516_25_K_22_1
Plik 289516_25_K_22_1 z AREA już istnieje
Plik 289516_25_K_22_1 z ZEROS już istnieje
i =  4
Z nagrania: 289540_23_K_13_2
Plik 289540_23_K_13_2 z AREA już istnieje
Plik 289540_23_K_13_2 z ZEROS już istnieje
i =  5
Z nagrania: 289540_23_K_18_3
Plik 289540_23_K_18_3 z AREA już istnieje
Plik 289540_23_K_18_3 z ZEROS już istnieje
i =  6
Z nagrania: 289540_23_K_22_4
Plik 289540_23_K_22_4 z AREA już istnieje
Plik 289540_23_K_22_4 z ZEROS już istnieje
i =  7
Z nagrania: 289540_23_K_9_1
Plik 289540_23_K_9_1 z AREA już istnieje
Plik 289540_23_K_9_1 z ZEROS już istnieje
i =  8
Z nagrania: 289550_23_K_18_2

In [6]:
# Liczenie parametrów zapisywanych do plików słów (w każdym pliku 18 wartości, kolejne osoby wypowiadające to samo słowo)

area_folder_name = path + '/' + 'Area_słowa' + '/' # tworzenie folderu na wyniki danego parametru
zeros_folder_name = path + '/' + 'Zeros_słowa' + '/'

for j in range(98):
    print(j)
    labels = []
    if j > 0:
        for a in range(0, j):
            labels.append('')
    area_word = []
    zeros_word = []
    zeros_cnt = 0
    slowo = []
    for i in range(len(list_of_names)):
        filename = list_of_names[i]
        file_wav = files_wav_list[i]
        file_txt = files_txt_list[i]
        textfile = pd.read_csv(path + '/' + file_txt, sep = '\t', header = None)
        labels.append(str(textfile[2][j]))
        if labels[j] == 'nan':
            labels[j] = 'NA'
        labels[j] = labels[j].upper()
        if i == 0:
            print(labels[j])
        person_path = path + '/' + labels[j]
        # Sprawdź, czy istnieje plik o danej nazwie
        if os.path.isfile(person_path + '/' + filename + '_' + '{0}.wav'.format(labels[j])) == True:
            new_filename = filename + '_' + labels[j] + '.wav'
            person = wave.open(person_path + '/' + new_filename, 'rb')
            # Jeśli ilość ramek nieparzysta, to nie zadziała, więc wtedy wczytujemy wszystkie ramki oprócz ostatniej
            if person.getnframes() % 2 != 0:
                person_frames = person.readframes(person.getnframes()-1)
            else:
                person_frames = person.readframes(person.getnframes())
            person_frames = np.frombuffer(person_frames, dtype=int)
            slowo.append(person_frames)
            if os.path.isdir(area_folder_name) == False:
                os.mkdir(area_folder_name)
            if os.path.isdir(zeros_folder_name) == False:
                os.mkdir(zeros_folder_name)
            area_word.append(simps(np.abs(slowo[i])))
            for m in range(1, len(slowo[i])):
                if (slowo[i][m] <= 0 and slowo[i][m-1] > 0) or (slowo[i][m] >= 0 and slowo[i][m-1] < 0):
                    zeros_cnt += 1
            zeros_word.append(zeros_cnt)
    area_txt_person_filename = area_folder_name + 'area_' + labels[j] + '.txt'
    zeros_txt_person_filename = zeros_folder_name + 'zeros_' + labels[j] + '.txt'
    if os.path.isfile(area_txt_person_filename) == False:
        np.savetxt(area_txt_person_filename, area_word)
        print('Zapisano jako:', area_txt_person_filename)
    else:
        print('Plik', labels[j], 'z AREA już istnieje') 
    if os.path.isfile(zeros_txt_person_filename) == False:
        np.savetxt(zeros_txt_person_filename, zeros_word)
        print('Zapisano jako:', zeros_txt_person_filename)
    elif os.path.isfile(zeros_txt_person_filename) == True:
        print('Plik', labels[j], 'z ZEROS już istnieje')             
            


0
OTWORZ
Plik OTWORZ z AREA już istnieje
Plik OTWORZ z ZEROS już istnieje
1
WLACZ
Plik WLACZ z AREA już istnieje
Plik WLACZ z ZEROS już istnieje
2
SWIATLO
Plik SWIATLO z AREA już istnieje
Plik SWIATLO z ZEROS już istnieje
3
TELEWIZOR
Plik TELEWIZOR z AREA już istnieje
Plik TELEWIZOR z ZEROS już istnieje
4
ZAMKNIJ
Plik ZAMKNIJ z AREA już istnieje
Plik ZAMKNIJ z ZEROS już istnieje
5
KUCHNI
Plik KUCHNI z AREA już istnieje
Plik KUCHNI z ZEROS już istnieje
6
NA
Plik NA z AREA już istnieje
Plik NA z ZEROS już istnieje
7
DRZWI
Plik DRZWI z AREA już istnieje
Plik DRZWI z ZEROS już istnieje
8
LAZIENCE
Plik LAZIENCE z AREA już istnieje
Plik LAZIENCE z ZEROS już istnieje
9
WYLACZ
Plik WYLACZ z AREA już istnieje
Plik WYLACZ z ZEROS już istnieje
10
W
Plik W z AREA już istnieje
Plik W z ZEROS już istnieje
11
OKNO
Plik OKNO z AREA już istnieje
Plik OKNO z ZEROS już istnieje
12
PIEKARNIK
Plik PIEKARNIK z AREA już istnieje
Plik PIEKARNIK z ZEROS już istnieje
13
ROLETY
Plik ROLETY z AREA już istnieje
Pl